Set MLflow tracker

In [2]:
import mlflow
# Provide uri and connect to your tracking server
mlflow.set_tracking_uri('http://127.0.0.1:5000/')

In [3]:
model_name = "prophet_forecasting"
model_version = 1
model_description = "This model forecasts the oil production based on the previous data."
version_description = "mlflow model version"

### MLflow Model Registry 

In [4]:
import time
from mlflow.tracking.client import MlflowClient
from mlflow.entities.model_registry.model_version_status import ModelVersionStatus
 
def wait_until_ready(model_name, model_version):
    client = MlflowClient()
    for _ in range(10):
        model_version_details = client.get_model_version(
        name=model_name,
        version=model_version,
        )
        status = ModelVersionStatus.from_string(model_version_details.status)
        print("Model status: %s" % ModelVersionStatus.to_string(status))
        if status == ModelVersionStatus.READY:
            break
        time.sleep(1)

In [5]:
wait_until_ready(model_name, model_version)

Model status: READY


In [6]:
from mlflow.tracking.client import MlflowClient
 
client = MlflowClient()
client.update_registered_model(
  name=model_name,
  description=model_description
)

<RegisteredModel: creation_timestamp=1677327553538, description='This model forecasts the oil production based on the previous data.', last_updated_timestamp=1677329971368, latest_versions=[<ModelVersion: creation_timestamp=1677327553627, current_stage='None', description='', last_updated_timestamp=1677327553627, name='prophet_forecasting', run_id='037a1a77df1940a3b82e71dfb87d87e4', run_link='', source='./mlflow/037a1a77df1940a3b82e71dfb87d87e4/artifacts/prophet_forecasting', status='READY', status_message='', tags={}, user_id='', version='1'>], name='prophet_forecasting', tags={}>

In [ ]:
client.update_model_version(
  name=model_name,
  version=model_version,
  description=version_description
)

In [7]:
client.transition_model_version_stage(
  name=model_name,
  version=model_version,
  stage='Production',
)

<ModelVersion: creation_timestamp=1677327553627, current_stage='Production', description='', last_updated_timestamp=1677330091639, name='prophet_forecasting', run_id='037a1a77df1940a3b82e71dfb87d87e4', run_link='', source='./mlflow/037a1a77df1940a3b82e71dfb87d87e4/artifacts/prophet_forecasting', status='READY', status_message='', tags={}, user_id='', version='1'>

In [8]:
model_version_details = client.get_model_version(
  name=model_name,
  version=model_version,
)
print("The current model stage is: '{stage}'".format(stage=model_version_details.current_stage))

The current model stage is: 'Production'


In [9]:
latest_version_info = client.get_latest_versions(model_name, stages=["Production"])
latest_production_version = latest_version_info[0].version
print("The latest production version of the model '%s' is '%s'." % (model_name, latest_production_version))

The latest production version of the model 'prophet_forecasting' is '1'.
